# Tensorflow Object Detection API and AWS Sagemaker

In [1]:
#faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.config

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [2]:
%%capture
%pip install tensorflow_io sagemaker -U

In [3]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [4]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::359346771395:role/service-role/AmazonSageMaker-ExecutionRole-20240516T125452


In [5]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://udacity-selfdriving-240516-1358/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [6]:
%%bash

# clone the repo and get the scripts
# git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
# cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
# cp docker/models/research/object_detection/model_main_tf2.py source_dir

In [7]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.

#image_name = 'tf2-object-detection'
#!sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [8]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

359346771395.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20240527120237


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [9]:
%%bash
pwd
rm -r /tmp/checkpoint
rm -r source_dir/checkpoint
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint

#wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
#tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

#ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8
#wget -O /tmp/ssdmobilenet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/ssdmobilenet.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint
# result out of memory with pipeline_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config

#Faster R-CNN ResNet152 V1 640x640
wget -O /tmp/fasterrcnn.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/fasterrcnn.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

#SSD ResNet50 V1 FPN 640x640 (RetinaNet50)
#wget -O /tmp/ssdresnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/ssdresnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

#Faster R-CNN ResNet50 V1 640x640
#wget -O /tmp/fasterrcnn50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
#tar -zxvf /tmp/fasterrcnn50.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint


/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training


--2024-06-06 13:48:59--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.167.207, 172.253.62.207, 172.253.115.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.167.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 470656289 (449M) [application/x-tar]
Saving to: ‘/tmp/fasterrcnn.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 11.6M 39s
    50K .......... .......... .......... .......... ..........  0% 22.9M 29s
   100K .......... .......... .......... .......... ..........  0% 21.7M 26s
   150K .......... .......... .......... .......... ..........  0% 22.7M 25s
   200K .......... .......... .......... .......... ..........  0% 53.1M 21s
   250K .......... .......... .......... .......... ..........  0% 25.8M 21s
   300K .......... .......... .......... .......

  5050K .......... .......... .......... .......... ..........  1% 71.1M 14s
  5100K .......... .......... .......... .......... ..........  1% 17.9M 14s
  5150K .......... .......... .......... .......... ..........  1%  368M 14s
  5200K .......... .......... .......... .......... ..........  1%  395M 14s
  5250K .......... .......... .......... .......... ..........  1%  375M 14s
  5300K .......... .......... .......... .......... ..........  1%  487M 14s
  5350K .......... .......... .......... .......... ..........  1%  208M 14s
  5400K .......... .......... .......... .......... ..........  1%  133M 14s
  5450K .......... .......... .......... .......... ..........  1% 84.3M 14s
  5500K .......... .......... .......... .......... ..........  1%  122M 14s
  5550K .......... .......... .......... .......... ..........  1% 81.4M 13s
  5600K .......... .......... .......... .......... ..........  1% 78.4M 13s
  5650K .......... .......... .......... .......... ..........  1% 74.0M 13s

 10400K .......... .......... .......... .......... ..........  2%  205M 9s
 10450K .......... .......... .......... .......... ..........  2%  100M 9s
 10500K .......... .......... .......... .......... ..........  2%  102M 9s
 10550K .......... .......... .......... .......... ..........  2%  105M 9s
 10600K .......... .......... .......... .......... ..........  2%  415M 9s
 10650K .......... .......... .......... .......... ..........  2%  108M 9s
 10700K .......... .......... .......... .......... ..........  2% 65.1M 9s
 10750K .......... .......... .......... .......... ..........  2%  270M 9s
 10800K .......... .......... .......... .......... ..........  2%  199M 9s
 10850K .......... .......... .......... .......... ..........  2%  137M 9s
 10900K .......... .......... .......... .......... ..........  2%  103M 9s
 10950K .......... .......... .......... .......... ..........  2%  173M 9s
 11000K .......... .......... .......... .......... ..........  2%  104M 9s
 11050K ....

 15800K .......... .......... .......... .......... ..........  3%  114M 7s
 15850K .......... .......... .......... .......... ..........  3%  181M 7s
 15900K .......... .......... .......... .......... ..........  3%  252M 7s
 15950K .......... .......... .......... .......... ..........  3%  209M 7s
 16000K .......... .......... .......... .......... ..........  3%  117M 7s
 16050K .......... .......... .......... .......... ..........  3%  123M 7s
 16100K .......... .......... .......... .......... ..........  3%  150M 7s
 16150K .......... .......... .......... .......... ..........  3%  157M 7s
 16200K .......... .......... .......... .......... ..........  3%  323M 7s
 16250K .......... .......... .......... .......... ..........  3% 87.5M 7s
 16300K .......... .......... .......... .......... ..........  3%  182M 7s
 16350K .......... .......... .......... .......... ..........  3% 2.47M 7s
 16400K .......... .......... .......... .......... ..........  3% 68.1M 7s
 16450K ....

...

442700K .......... .......... .......... .......... .......... 96%  183M 0s
442750K .......... .......... .......... .......... .......... 96%  161M 0s
442800K .......... .......... .......... .......... .......... 96%  162M 0s
442850K .......... .......... .......... .......... .......... 96%  233M 0s
442900K .......... .......... .......... .......... .......... 96%  166M 0s
442950K .......... .......... .......... .......... .......... 96%  191M 0s
443000K .......... .......... .......... .......... .......... 96%  152M 0s
443050K .......... .......... .......... .......... .......... 96%  125M 0s
443100K .......... .......... .......... .......... .......... 96%  187M 0s
443150K .......... .......... .......... .......... .......... 96%  162M 0s
443200K .......... .......... .......... .......... .......... 96%  148M 0s
443250K .......... .......... .......... .......... .......... 96%  199M 0s
443300K .......... .......... .......... .......... .......... 96%  211M 0s
443350K ....

448100K .......... .......... .......... .......... .......... 97%  381M 0s
448150K .......... .......... .......... .......... .......... 97%  355M 0s
448200K .......... .......... .......... .......... .......... 97%  322M 0s
448250K .......... .......... .......... .......... .......... 97%  348M 0s
448300K .......... .......... .......... .......... .......... 97%  366M 0s
448350K .......... .......... .......... .......... .......... 97%  372M 0s
448400K .......... .......... .......... .......... .......... 97%  314M 0s
448450K .......... .......... .......... .......... .......... 97%  368M 0s
448500K .......... .......... .......... .......... .......... 97%  350M 0s
448550K .......... .......... .......... .......... .......... 97%  372M 0s
448600K .......... .......... .......... .......... .......... 97%  317M 0s
448650K .......... .......... .......... .......... .......... 97%  354M 0s
448700K .......... .......... .......... .......... .......... 97%  299M 0s
448750K ....

453500K .......... .......... .......... .......... .......... 98%  345M 0s
453550K .......... .......... .......... .......... .......... 98%  354M 0s
453600K .......... .......... .......... .......... .......... 98%  300M 0s
453650K .......... .......... .......... .......... .......... 98%  449M 0s
453700K .......... .......... .......... .......... .......... 98%  410M 0s
453750K .......... .......... .......... .......... .......... 98%  376M 0s
453800K .......... .......... .......... .......... .......... 98%  322M 0s
453850K .......... .......... .......... .......... .......... 98%  405M 0s
453900K .......... .......... .......... .......... .......... 98%  410M 0s
453950K .......... .......... .......... .......... .......... 98%  334M 0s
454000K .......... .......... .......... .......... .......... 98%  392M 0s
454050K .......... .......... .......... .......... .......... 98%  325M 0s
454100K .......... .......... .......... .......... .......... 98%  322M 0s
454150K ....

458900K .......... .......... .......... .......... .......... 99%  366M 0s
458950K .......... .......... .......... .......... .......... 99%  450M 0s
459000K .......... .......... .......... .......... .......... 99%  317M 0s
459050K .......... .......... .......... .......... .......... 99%  299M 0s
459100K .......... .......... .......... .......... .......... 99%  398M 0s
459150K .......... .......... .......... .......... .......... 99%  402M 0s
459200K .......... .......... .......... .......... .......... 99%  267M 0s
459250K .......... .......... .......... .......... .......... 99%  354M 0s
459300K .......... .......... .......... .......... .......... 99%  371M 0s
459350K .......... .......... .......... .......... .......... 99%  307M 0s
459400K .......... .......... .......... .......... .......... 99%  303M 0s
459450K .......... .......... .......... .......... .......... 99%  386M 0s
459500K .......... .......... .......... .......... .......... 99%  413M 0s
459550K ....

faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [10]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.xlarge', # ml.g5.xlarge
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2024-06-06-13-49-09-344


2024-06-06 13:49:21 Starting - Starting the training job...
2024-06-06 13:49:36 Starting - Preparing the instances for training...
2024-06-06 13:50:07 Downloading - Downloading input data...
2024-06-06 13:50:27 Downloading - Downloading the training image.........
2024-06-06 13:52:03 Training - Training image download completed. Training in progress..2024-06-06 13:52:18,062 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 13:52:18,100 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 13:52:18,136 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 13:52:18,151 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "current_insta

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0606 13:52:31.070716 140065472317248 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0606 13:52:33.668576 140065472317248 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
/usr/local/lib/python3.8/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/Fa

==EVALUATING THE MODEL==
W0606 13:56:17.786066 139851530557248 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0606 13:56:17.786251 139851530557248 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0606 13:56:17.786344 139851530557248 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0606 13:56:17.786538 139851530557248 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0606 13:56:17.786729 139851530557248 model_lib_v2.py:1106] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:Reading unweighted datasets: ['/opt/ml/input/data/val/*.tfrecord']
I0606 13:56:18.168549 139851530557248 dataset_builder.py:162] Reading unweighted datasets: ['/o


2024-06-06 13:58:12 Uploading - Uploading generated training model
2024-06-06 13:58:12 Failed - Training job failed


UnexpectedStatusException: Error for Training job tf2-object-detection-2024-06-06-13-49-09-344: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage ""
Command "/bin/sh -c ./run_training.sh --model_dir /opt/training --num_train_steps 2000 --pipeline_config_path faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.config --sample_1_of_n_eval_examples 1", exit code: 1

In [ ]:
# DEBUG / TROUBLESHOOT
#ls /tmp/checkpoint
#mkdir source_dir/checkpoint

import os

# Get the current working directory
current_directory = os.getcwd()
print(current_directory)

# Extract the directory name
directory_name = os.path.basename(current_directory)

# Print the directory name
print(directory_name)

files = [f for f in os.listdir('/home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir/checkpoint')]
for f in files:
    print(f)
    # do something

print("\n/opt")
files = [f for f in os.listdir('/opt/')]
for f in files:
    print(f)
    # do something

    
# /home/ec2-user/SageMaker/cd2688-object-detection-in-urban-environment-project/1_model_training/source_dir

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [ ]:
# your write-up goes here.